# Notebook 1: Tiền xử lý và Chuẩn bị Dữ liệu (Phiên bản Nâng cao)
## Dự đoán từ tiếp theo - Next Word Prediction

Notebook này thực hiện:
1. Tải dữ liệu từ VTSNLP dataset (10,000 dòng)
2. Tiền xử lý văn bản theo cấu trúc phân cấp: **Văn bản → Đoạn văn → Câu → Từ/Cụm từ**
3. Loại bỏ ký tự đặc biệt và làm sạch dữ liệu kỹ lưỡng
4. Tokenization với underthesea
5. Xây dựng vocabulary và encoding
6. Lưu dữ liệu đã tiền xử lý

## 1. Import thư viện

In [1]:
# Cài đặt các thư viện cần thiết
# !pip install datasets underthesea numpy

In [2]:
import re
import pickle
import numpy as np
from datasets import load_dataset
from underthesea import word_tokenize, sent_tokenize
from collections import Counter, defaultdict
import os

print("Thư viện đã được import thành công!")

c:\Users\thanh\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Thư viện đã được import thành công!


## 2. Tải dữ liệu từ VTSNLP

In [3]:
# Tải dataset
print("Đang tải dữ liệu từ VTSNLP...")
ds = load_dataset("VTSNLP/vietnamese_curated_dataset")
print(f"Dataset đã tải: {ds}")
print(f"\nCấu trúc: {ds['train'].features}")

Đang tải dữ liệu từ VTSNLP...
Dataset đã tải: DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'domain'],
        num_rows: 12169131
    })
})

Cấu trúc: {'text': Value('string'), 'id': Value('string'), 'domain': Value('string')}


In [4]:
# Lấy 10,000 dòng dữ liệu
NUM_SAMPLES = 500000
train_data = ds['train'].select(range(min(NUM_SAMPLES, len(ds['train']))))

print(f"Số lượng mẫu: {len(train_data)}")
print(f"\nVí dụ mẫu đầu tiên:")
print(train_data[0]['text'][:300])

Số lượng mẫu: 500000

Ví dụ mẫu đầu tiên:
Internet Society hay ISOC là một tổ chức quốc tế hoạt động phi lợi nhuận, phi chính phủ và bao gồm các thành viên có trình độ chuyên ngành. Tổ chức này chú trọng đến: tiêu chuẩn, giáo dục và các vấn đề về chính sách. Với trên 145 tổ chức thành viên và 65.000 thành viên cá nhân, ISOC bao gồm những co


In [5]:
# Trích xuất văn bản
texts = []
for item in train_data:
    if 'text' in item:
        texts.append(item['text'])
    elif 'content' in item:
        texts.append(item['content'])
    elif 'sentence' in item:
        texts.append(item['sentence'])

print(f"Đã trích xuất {len(texts)} văn bản")

Đã trích xuất 500000 văn bản


## 3. Tiền xử lý văn bản - Loại bỏ ký tự đặc biệt

In [6]:
def clean_text_advanced(text):
    """
    Làm sạch văn bản nâng cao:
    - Loại bỏ URL, email, số điện thoại
    - Loại bỏ các ký tự đặc biệt không cần thiết
    - Loại bỏ số đứng riêng
    - Chuẩn hóa dấu câu
    - Chuẩn hóa khoảng trắng
    - Chuyển về chữ thường
    """
    if not isinstance(text, str):
        return ""
    
    # Chuyển về chữ thường
    text = text.lower()
    
    # Loại bỏ URL
    text = re.sub(r'http\S+|www\.\S+', '', text)
    
    # Loại bỏ email
    text = re.sub(r'\S+@\S+', '', text)
    
    # Loại bỏ số điện thoại (dạng 0xxx-xxx-xxx hoặc 0xxxxxxxxx)
    text = re.sub(r'\b0\d{9,10}\b', '', text)
    text = re.sub(r'\b\d{3}[-.]\d{3}[-.]\d{4}\b', '', text)
    
    # Loại bỏ các ký tự đặc biệt, chỉ giữ:
    # - Chữ cái tiếng Việt (có dấu)
    # - Số (sẽ lọc riêng sau)
    # - Dấu câu cơ bản: . , ! ? ; :
    text = re.sub(
        r'[^a-záàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵđ0-9\s.,!?;:\-]',
        '',
        text
    )
    
    # Loại bỏ số đứng riêng (không phải là phần của từ)
    text = re.sub(r'\b\d+\b', '', text)
    
    # Loại bỏ các ký tự đơn lẻ (trừ các từ có nghĩa như "ở", "ý", "á")
    # text = re.sub(r'\b[a-z]\b', '', text)
    
    # Chuẩn hóa dấu câu - loại bỏ dấu câu trùng lặp
    text = re.sub(r'\.{2,}', '.', text)  # ... -> .
    text = re.sub(r',{2,}', ',', text)   # ,, -> ,
    text = re.sub(r'!{2,}', '!', text)   # !! -> !
    text = re.sub(r'\?{2,}', '?', text)  # ?? -> ?
    text = re.sub(r';{2,}', ';', text)   # ;; -> ;
    text = re.sub(r':{2,}', ':', text)   # :: -> :
    text = re.sub(r'-{2,}', '-', text)   # -- -> -
    
    # Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text)
    
    # Loại bỏ khoảng trắng trước dấu câu
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)
    
    # Thêm khoảng trắng sau dấu câu nếu chưa có
    text = re.sub(r'([.,!?;:])([^\s])', r'\1 \2', text)
    
    # Loại bỏ khoảng trắng ở đầu và cuối
    text = text.strip()
    
    # Loại bỏ dấu câu ở đầu câu
    text = re.sub(r'^[.,!?;:\-]+', '', text)
    
    return text

# Test hàm clean_text_advanced
print("=" * 60)
print("TEST HÀM CLEAN TEXT")
print("=" * 60)

test_samples = [
    "Xin chào!!! Đây là VÍ DỤ 123... http://example.com",
    "Email: test@email.com, SĐT: 0912345678",
    "Năm   2024...   Việt Nam!!!!",
    "Giá    100,000đ -- khuyến mãi 50%"
]

for sample in test_samples:
    cleaned = clean_text_advanced(sample)
    print(f"\nTrước: {sample}")
    print(f"Sau:  {cleaned}")

TEST HÀM CLEAN TEXT

Trước: Xin chào!!! Đây là VÍ DỤ 123... http://example.com
Sau:  xin chào! đây là ví dụ.

Trước: Email: test@email.com, SĐT: 0912345678
Sau:  email: sđt:

Trước: Năm   2024...   Việt Nam!!!!
Sau:  năm. việt nam!

Trước: Giá    100,000đ -- khuyến mãi 50%
Sau:  giá, 000đ - khuyến mãi


In [7]:
# Làm sạch toàn bộ dữ liệu
print("\nĐang làm sạch dữ liệu...")
cleaned_texts = []

for i, text in enumerate(texts):
    if i % 1000 == 0:
        print(f"Đã xử lý {i}/{len(texts)} văn bản")
    
    cleaned = clean_text_advanced(text)
    # Chỉ giữ văn bản có độ dài >= 10 ký tự
    if len(cleaned) >= 10:
        cleaned_texts.append(cleaned)

print(f"\nSố văn bản sau khi làm sạch: {len(cleaned_texts)}")
print(f"\nVí dụ văn bản sau khi làm sạch:")
print(cleaned_texts[0][:300])


Đang làm sạch dữ liệu...
Đã xử lý 0/500000 văn bản
Đã xử lý 1000/500000 văn bản
Đã xử lý 2000/500000 văn bản
Đã xử lý 3000/500000 văn bản
Đã xử lý 4000/500000 văn bản
Đã xử lý 5000/500000 văn bản
Đã xử lý 6000/500000 văn bản
Đã xử lý 7000/500000 văn bản
Đã xử lý 8000/500000 văn bản
Đã xử lý 9000/500000 văn bản
Đã xử lý 10000/500000 văn bản
Đã xử lý 11000/500000 văn bản
Đã xử lý 12000/500000 văn bản
Đã xử lý 13000/500000 văn bản
Đã xử lý 14000/500000 văn bản
Đã xử lý 15000/500000 văn bản
Đã xử lý 16000/500000 văn bản
Đã xử lý 17000/500000 văn bản
Đã xử lý 18000/500000 văn bản
Đã xử lý 19000/500000 văn bản
Đã xử lý 20000/500000 văn bản
Đã xử lý 21000/500000 văn bản
Đã xử lý 22000/500000 văn bản
Đã xử lý 23000/500000 văn bản
Đã xử lý 24000/500000 văn bản
Đã xử lý 25000/500000 văn bản
Đã xử lý 26000/500000 văn bản
Đã xử lý 27000/500000 văn bản
Đã xử lý 28000/500000 văn bản
Đã xử lý 29000/500000 văn bản
Đã xử lý 30000/500000 văn bản
Đã xử lý 31000/500000 văn bản
Đã xử lý 32000/500000 văn b

## 4. Xử lý phân cấp: Văn bản → Đoạn văn → Câu → Từ

In [8]:
def split_into_paragraphs(text):
    """
    Tách văn bản thành các đoạn văn
    Đoạn văn được phân tách bởi xuống dòng (\n)
    """
    # Tách theo dấu xuống dòng
    paragraphs = text.split('\n')
    
    # Loại bỏ đoạn văn rỗng
    paragraphs = [p.strip() for p in paragraphs if p.strip()]
    
    return paragraphs

def split_into_sentences(text):
    """
    Tách văn bản/đoạn văn thành các câu
    Sử dụng sent_tokenize của underthesea
    """
    try:
        sentences = sent_tokenize(text)
        # Loại bỏ câu rỗng và quá ngắn (< 5 ký tự)
        sentences = [s.strip() for s in sentences if len(s.strip()) >= 5]
        return sentences
    except:
        # Nếu lỗi, tách đơn giản bằng dấu câu
        sentences = re.split(r'[.!?]+', text)
        return [s.strip() for s in sentences if len(s.strip()) >= 5]

def tokenize_sentence(sentence):
    """
    Tách câu thành các từ/cụm từ
    Sử dụng word_tokenize của underthesea
    """
    try:
        tokens = word_tokenize(sentence, format="text")
        return tokens.split()
    except:
        # Nếu lỗi, dùng split đơn giản
        return sentence.split()

# Test các hàm phân cấp
print("=" * 60)
print("TEST XỬ LÝ PHÂN CẤP")
print("=" * 60)

sample_text = """Đây là đoạn văn đầu tiên. Nó có nhiều câu.

Đây là đoạn văn thứ hai. Tôi đi học bằng xe buýt. Hôm nay trời đẹp.

Đoạn văn cuối cùng."""

# Tách đoạn văn
paragraphs = split_into_paragraphs(sample_text)
print(f"\nSố đoạn văn: {len(paragraphs)}")
for i, para in enumerate(paragraphs, 1):
    print(f"Đoạn {i}: {para}")

# Tách câu từ đoạn đầu tiên
print(f"\n\nTách câu từ đoạn 1:")
sentences = split_into_sentences(paragraphs[0])
for i, sent in enumerate(sentences, 1):
    print(f"Câu {i}: {sent}")

# Tokenize câu đầu tiên
if sentences:
    print(f"\n\nTokenize câu 1:")
    tokens = tokenize_sentence(sentences[0])
    print(f"Tokens: {tokens}")

TEST XỬ LÝ PHÂN CẤP

Số đoạn văn: 3
Đoạn 1: Đây là đoạn văn đầu tiên. Nó có nhiều câu.
Đoạn 2: Đây là đoạn văn thứ hai. Tôi đi học bằng xe buýt. Hôm nay trời đẹp.
Đoạn 3: Đoạn văn cuối cùng.


Tách câu từ đoạn 1:
Câu 1: Đây là đoạn văn đầu tiên.
Câu 2: Nó có nhiều câu.


Tokenize câu 1:
Tokens: ['Đây', 'là', 'đoạn', 'văn', 'đầu_tiên', '.']


In [ ]:
# Xử lý toàn bộ dữ liệu theo cấu trúc phân cấp
print("\n" + "=" * 60)
print("XỬ LÝ TOÀN BỘ DỮ LIỆU THEO CẤU TRÚC PHÂN CẤP")
print("=" * 60)

all_sentences = []
all_tokens = []

for i, text in enumerate(cleaned_texts):
    if i % 1000 == 0:
        print(f"Đang xử lý văn bản {i}/{len(cleaned_texts)}...")
    
    # Bước 1: Tách thành đoạn văn
    paragraphs = split_into_paragraphs(text)
    
    # Bước 2: Tách mỗi đoạn văn thành câu
    for paragraph in paragraphs:
        sentences = split_into_sentences(paragraph)
        
        # Bước 3: Tokenize mỗi câu thành từ/cụm từ
        for sentence in sentences:
            tokens = tokenize_sentence(sentence)
            
            # Loại bỏ các token là dấu câu đơn thuần
            tokens = [t for t in tokens if not re.match(r'^[.,!?;:\-]+$', t)]
            
            # Chỉ giữ câu có ít nhất 3 từ
            if len(tokens) >= 3:
                all_sentences.append(sentence)
                all_tokens.append(tokens)

print(f"\nTổng số câu: {len(all_sentences)}")
print(f"Tổng số sequences tokens: {len(all_tokens)}")
print(f"\nVí dụ 5 câu đầu tiên:")
for i in range(min(5, len(all_sentences))):
    print(f"\nCâu {i+1}: {all_sentences[i]}")
    print(f"Tokens: {all_tokens[i][:15]}...")


XỬ LÝ TOÀN BỘ DỮ LIỆU THEO CẤU TRÚC PHÂN CẤP
Đang xử lý văn bản 0/499997...
Đang xử lý văn bản 1000/499997...
Đang xử lý văn bản 2000/499997...
Đang xử lý văn bản 3000/499997...
Đang xử lý văn bản 4000/499997...
Đang xử lý văn bản 5000/499997...
Đang xử lý văn bản 6000/499997...
Đang xử lý văn bản 7000/499997...
Đang xử lý văn bản 8000/499997...
Đang xử lý văn bản 9000/499997...
Đang xử lý văn bản 10000/499997...
Đang xử lý văn bản 11000/499997...
Đang xử lý văn bản 12000/499997...


## 5. Xây dựng Vocabulary

In [ ]:
# Đếm tần suất từ
print("Đang đếm tần suất từ...")
word_counts = Counter()

for tokens in all_tokens:
    word_counts.update(tokens)

print(f"\nTổng số từ duy nhất: {len(word_counts)}")
print(f"\n15 từ phổ biến nhất:")
for word, count in word_counts.most_common(15):
    print(f"  {word}: {count}")

print(f"\n15 từ ít phổ biến nhất (xuất hiện 1 lần):")
rare_words = [(w, c) for w, c in word_counts.items() if c == 1]
print(f"Số từ xuất hiện 1 lần: {len(rare_words)}")
for word, count in list(rare_words)[:15]:
    print(f"  {word}: {count}")

In [ ]:
# Tạo vocabulary - chỉ giữ các từ xuất hiện ít nhất MIN_WORD_FREQ lần
MIN_WORD_FREQ = 2
vocab = ['<PAD>', '<UNK>', '<START>', '<END>']  # Special tokens

# Thêm các từ có tần suất >= MIN_WORD_FREQ
for word, count in word_counts.most_common():
    if count >= MIN_WORD_FREQ:
        vocab.append(word)

print(f"\nKích thước vocabulary (min_freq={MIN_WORD_FREQ}): {len(vocab)}")

# Tạo mapping word <-> index
word2idx = {word: idx for idx, word in enumerate(vocab)}
idx2word = {idx: word for word, idx in word2idx.items()}

print(f"\nVí dụ word2idx:")
test_words = ['<PAD>', '<UNK>', 'tôi', 'là', 'đi', 'học', 'việt_nam']
for word in test_words:
    if word in word2idx:
        print(f"  {word}: {word2idx[word]}")
    else:
        print(f"  {word}: <NOT IN VOCAB>")

## 6. Encoding và tạo Training Sequences

In [ ]:
def encode_tokens(tokens, word2idx):
    """
    Chuyển đổi tokens thành sequence of indices
    """
    return [word2idx.get(token, word2idx['<UNK>']) for token in tokens]

# Encode toàn bộ dữ liệu
print("Đang encode dữ liệu...")
encoded_sequences = []

for tokens in all_tokens:
    encoded = encode_tokens(tokens, word2idx)
    if len(encoded) >= 3:  # Cần ít nhất 3 từ
        encoded_sequences.append(encoded)

print(f"Số sequences đã encode: {len(encoded_sequences)}")
print(f"\nVí dụ:")
print(f"Tokens: {all_tokens[0][:10]}")
print(f"Encoded: {encoded_sequences[0][:10]}")

In [ ]:
def create_training_sequences(encoded_sequences, max_sequence_len=10):
    """
    Tạo các cặp (input_sequence, next_word) cho training
    """
    input_sequences = []
    
    for encoded_seq in encoded_sequences:
        for i in range(1, len(encoded_seq)):
            n = min(i, max_sequence_len)
            input_seq = encoded_seq[i-n:i]
            target_word = encoded_seq[i]
            input_sequences.append((input_seq, target_word))
    
    return input_sequences

# Tạo training sequences
MAX_SEQ_LEN = 10
print(f"\nĐang tạo training sequences (max_len={MAX_SEQ_LEN})...")
training_sequences = create_training_sequences(encoded_sequences, MAX_SEQ_LEN)

print(f"Tổng số training pairs: {len(training_sequences)}")
print(f"\nVí dụ 5 sequences:")
for i in range(min(5, len(training_sequences))):
    seq, target = training_sequences[i]
    seq_words = [idx2word[idx] for idx in seq]
    target_word = idx2word[target]
    print(f"  {seq_words} -> {target_word}")

## 7. Padding Sequences

In [ ]:
def pad_sequences(sequences, max_len, pad_value=0):
    """
    Padding sequences về cùng độ dài
    """
    padded = []
    for seq in sequences:
        if len(seq) < max_len:
            padded_seq = [pad_value] * (max_len - len(seq)) + seq
        else:
            padded_seq = seq[-max_len:]
        padded.append(padded_seq)
    return np.array(padded)

# Tách input và target
X_sequences = [seq for seq, _ in training_sequences]
y_targets = np.array([target for _, target in training_sequences])

# Padding
X_padded = pad_sequences(X_sequences, MAX_SEQ_LEN, pad_value=word2idx['<PAD>'])

print(f"Shape của X: {X_padded.shape}")
print(f"Shape của y: {y_targets.shape}")
print(f"\nVí dụ sau khi padding:")
print(f"X[0]: {X_padded[0]}")
print(f"y[0]: {y_targets[0]} ({idx2word[y_targets[0]]})")

## 8. Thống kê dữ liệu

In [ ]:
print("=" * 70)
print("THỐNG KÊ DỮ LIỆU SAU TIỀN XỬ LÝ (PHIÊN BẢN NÂNG CAO)")
print("=" * 70)
print(f"\n📝 Dữ liệu gốc:")
print(f"  - Số văn bản ban đầu: {len(texts)}")
print(f"  - Số văn bản sau làm sạch: {len(cleaned_texts)}")

print(f"\n📊 Xử lý phân cấp:")
print(f"  - Tổng số câu: {len(all_sentences)}")
print(f"  - Tổng số sequences tokens: {len(all_tokens)}")
print(f"  - Trung bình {len(all_tokens)/len(cleaned_texts):.1f} câu/văn bản")

print(f"\n📖 Vocabulary:")
print(f"  - Tổng số từ duy nhất: {len(word_counts)}")
print(f"  - Kích thước vocabulary (min_freq={MIN_WORD_FREQ}): {len(vocab)}")
print(f"  - Tỷ lệ giữ lại: {len(vocab)/len(word_counts)*100:.1f}%")

print(f"\n🎯 Training data:")
print(f"  - Số sequences đã encode: {len(encoded_sequences)}")
print(f"  - Tổng số training pairs: {len(training_sequences)}")
print(f"  - Max sequence length: {MAX_SEQ_LEN}")

print(f"\n📦 Shape của dữ liệu:")
print(f"  - X shape: {X_padded.shape}")
print(f"  - y shape: {y_targets.shape}")
print("=" * 70)

## 9. Lưu dữ liệu đã tiền xử lý

In [ ]:
# Tạo thư mục
os.makedirs('../data/processed', exist_ok=True)

print("Đang lưu dữ liệu...")

# 1. Vocabulary
with open('../data/processed/vocabulary.pkl', 'wb') as f:
    pickle.dump({
        'vocab': vocab,
        'word2idx': word2idx,
        'idx2word': idx2word,
        'vocab_size': len(vocab),
        'word_counts': dict(word_counts.most_common(1000))  # Top 1000 từ
    }, f)
print("✓ Đã lưu vocabulary.pkl")

# 2. Training data
with open('../data/processed/training_data.pkl', 'wb') as f:
    pickle.dump({
        'X': X_padded,
        'y': y_targets,
        'max_seq_len': MAX_SEQ_LEN
    }, f)
print("✓ Đã lưu training_data.pkl")

# 3. Tokenized texts (cho n-gram)
with open('../data/processed/tokenized_texts.pkl', 'wb') as f:
    pickle.dump(all_tokens, f)
print("✓ Đã lưu tokenized_texts.pkl")

# 4. Sentences (cho phân tích)
with open('../data/processed/sentences.pkl', 'wb') as f:
    pickle.dump(all_sentences[:10000], f)  # Lưu 10000 câu đầu
print("✓ Đã lưu sentences.pkl")

# 5. Config
config = {
    'num_documents': len(texts),
    'num_cleaned_documents': len(cleaned_texts),
    'num_sentences': len(all_sentences),
    'num_sequences': len(all_tokens),
    'vocab_size': len(vocab),
    'max_seq_len': MAX_SEQ_LEN,
    'min_word_freq': MIN_WORD_FREQ,
    'num_training_pairs': len(training_sequences),
    'preprocessing_method': 'hierarchical',  # Văn bản -> Đoạn -> Câu -> Từ
}
with open('../data/processed/config.pkl', 'wb') as f:
    pickle.dump(config, f)
print("✓ Đã lưu config.pkl")

print("\n" + "="*70)
print("HOÀN THÀNH TIỀN XỬ LÝ DỮ LIỆU (PHIÊN BẢN NÂNG CAO)!")
print("="*70)
print("\n✅ Đã thực hiện:")
print("  1. Loại bỏ ký tự đặc biệt kỹ lưỡng")
print("  2. Xử lý phân cấp: Văn bản → Đoạn văn → Câu → Từ/Cụm từ")
print("  3. Tokenization với underthesea")
print("  4. Xây dựng vocabulary với filtering")
print("  5. Encoding và padding sequences")
print("\n📁 Các file đã lưu trong: ../data/processed/")
print("  - vocabulary.pkl")
print("  - training_data.pkl")
print("  - tokenized_texts.pkl")
print("  - sentences.pkl")
print("  - config.pkl")
print("\n➡️  Bạn có thể chuyển sang notebook 02 để training mô hình!")